In [103]:
import requests

# URL base da API
BASE_URL = "https://metadata.idl.ucsf.edu/solr/ltdl3/query"

def get_total_documents(max_pages=1):
    """
    Obtém a quantidade total de documentos disponíveis na API Solr, 
    aplicando um limite de páginas.

    Args:
        max_pages (int): Número máximo de páginas para filtrar os documentos. Padrão: 1.

    Returns:
        int: Número total de documentos disponíveis com o filtro aplicado.
    """
    # Construção da query com filtros
    query_parts = []
    query_parts.append(f"pages:[* TO {max_pages}]")  # Filtro de número máximo de páginas
    query = " AND ".join(query_parts) if query_parts else "*:*"  # Query final

    # Parâmetros da requisição
    params = {
        "q": query,         # Query construída manualmente
        "wt": "json",       # Resposta em formato JSON
        "rows": 0           # Não retorna documentos, apenas a contagem
    }

    # Requisição à API
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        # Extrai o número total de documentos
        total_documents = response.json().get("response", {}).get("numFound", 0)
        return total_documents
    else:
        # Trata erros na requisição
        print(f"Erro {response.status_code} ao consultar o total de documentos.")
        print(f"Mensagem de erro: {response.text}")
        return None


In [104]:
# Exemplo de uso
total_documents = get_total_documents()
if total_documents is not None:
    print(f"Quantidade total de documentos disponíveis com até 1 página: {total_documents}")
else:
    print("Falha ao obter o total de documentos.")

# Para consultar com um limite diferente de páginas
total_documents_5_pages = get_total_documents(max_pages=5)
if total_documents_5_pages is not None:
    print(f"Quantidade total de documentos disponíveis com até 5 páginas: {total_documents_5_pages}")
else:
    print("Falha ao obter o total de documentos.")

Quantidade total de documentos disponíveis com até 1 página: 10435636
Quantidade total de documentos disponíveis com até 5 páginas: 17356073


In [105]:
import math

def calculate_iterations(total_documents, max_results):
    """
    Calcula o número de iterações necessárias para processar todos os documentos.

    Args:
        total_documents (int): O número total de documentos disponíveis na API.
        max_results (int): O número máximo de documentos por lote (tamanho do lote).

    Returns:
        int: O número total de iterações necessárias.
    """
    if max_results <= 0:
        raise ValueError("max_results deve ser maior que zero.")
    if total_documents <= 0:
        raise ValueError("total_documents deve ser maior que zero.")
    
    # Calcula o número total de iterações
    iterations = math.ceil(total_documents / max_results)
    return iterations

# Exemplo de uso
try:
    # total_documents = total_documents  # Número total de documentos (exemplo da API)
    max_results = 100  # Tamanho do lote (exemplo)

    iterations = calculate_iterations(total_documents, max_results)
    print(f"Para processar {total_documents} documentos com lotes de {max_results}, serão necessárias {iterations} iterações.")
except ValueError as e:
    print(f"Erro: {e}")

Para processar 10435636 documentos com lotes de 100, serão necessárias 104357 iterações.


In [88]:
import requests
import sqlite3
import time
from datetime import datetime

# URL base da API
BASE_URL = "https://metadata.idl.ucsf.edu/solr/ltdl3/query"
DB_PATH = "document_comparisons.db"

def initialize_database():
    """
    Cria ou atualiza a tabela Collections_Types no banco de dados.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Criação ou atualização da tabela
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Collections_Types (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        collection TEXT NOT NULL,
        type TEXT NOT NULL,
        count INTEGER NOT NULL,
        cursorMark TEXT,
        UNIQUE(collection, type)
    )
    """)
    conn.commit()
    conn.close()


In [89]:
def save_collection_type(collection, doc_type, count, cursor_mark):
    """
    Salva ou atualiza uma combinação collection/type com sua contagem de documentos e o cursorMark.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    try:
        cursor.execute("""
        INSERT INTO Collections_Types (collection, type, count, cursorMark)
        VALUES (?, ?, ?, ?)
        ON CONFLICT(collection, type) DO UPDATE SET 
            count = count + excluded.count,
            cursorMark = excluded.cursorMark;
        """, (collection, doc_type, count, cursor_mark))
    except sqlite3.IntegrityError:
        pass  # Evita duplicatas

    conn.commit()
    conn.close()


In [108]:
def fetch_combinations(max_pages=1, total_documents=10435636):
    """
    Busca as combinações de collections e types diretamente em lotes usando cursorMark
    e continua a partir de onde parou, baseado nos dados já armazenados.
    
    Parâmetros:
        max_pages (int): Número máximo de páginas para filtrar os documentos na API (padrão: 1).
    """
    start_time = datetime.now()
    max_results = 100
    total_documents = total_documents  # Substitua pelo número real, se disponível

    # Conectar ao banco de dados
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Recuperar o último cursorMark e total de registros processados
    cursor.execute("SELECT cursorMark FROM Collections_Types ORDER BY id DESC LIMIT 1")
    last_cursor_mark = cursor.fetchone()
    cursor_mark = last_cursor_mark[0] if last_cursor_mark else "*"  # Define o ponto de partida

    cursor.execute("SELECT SUM(count) FROM Collections_Types")
    total_processed = cursor.fetchone()[0] or 0  # Total de registros já processados

    print(f"Iniciando processamento a partir do cursorMark: {cursor_mark}")
    print(f"Progresso anterior: {total_processed} documentos processados ({(total_processed / total_documents) * 100:.2f}%).")

    # Processar os dados
    while True:
        # Construção da query com filtros
        query_parts = [f"pages:[* TO {max_pages}]"]  # Filtro de número máximo de páginas
        query = " AND ".join(query_parts) if query_parts else "*:*"

        params = {
            "q": query,               # Query construída manualmente
            "fl": "collection,type",  # Retorna apenas os campos desejados
            "wt": "json",             # Formato da resposta
            "rows": max_results,      # Tamanho do lote
            "cursorMark": cursor_mark,  # Paginação com cursor
            "sort": "id asc"          # Ordenação necessária para usar cursorMark
        }

        response = requests.get(BASE_URL, params=params)

        if response.status_code == 200:
            data = response.json()
            docs = data.get("response", {}).get("docs", [])
            next_cursor_mark = data.get("nextCursorMark", cursor_mark)

            if not docs:
                break  # Sai do loop se não houver mais documentos

            # Processa as combinações de collection e type
            counts = {}
            for doc in docs:
                collections = doc.get("collection")
                doc_types = doc.get("type")

                # Trata collection e type como listas
                if not isinstance(collections, list):
                    collections = [collections]
                if not isinstance(doc_types, list):
                    doc_types = [doc_types]

                # Conta as combinações
                for collection in collections:
                    for doc_type in doc_types:
                        if collection and doc_type:
                            key = (collection, doc_type)
                            counts[key] = counts.get(key, 0) + 1

            # Salva cada combinação no banco de dados com o cursorMark
            for (collection, doc_type), count in counts.items():
                try:
                    cursor.execute("""
                        INSERT INTO Collections_Types (collection, type, count, cursorMark)
                        VALUES (?, ?, ?, ?)
                        ON CONFLICT(collection, type) DO UPDATE SET
                            count = count + excluded.count,
                            cursorMark = excluded.cursorMark
                    """, (collection, doc_type, count, cursor_mark))
                except sqlite3.Error as e:
                    print(f"Erro ao salvar no banco de dados: {e}")
            conn.commit()

            # Atualiza o total de registros processados
            total_processed += len(docs)

            # Atualiza progresso no terminal
            elapsed_time = datetime.now() - start_time
            percent_complete = (total_processed / total_documents) * 100
            print(f"\rProcessando... Cursor atual: {cursor_mark}, "
                  f"Documentos processados: {total_processed}, "
                  f"Progresso: {percent_complete:.2f}%, "
                  f"Tempo decorrido: {elapsed_time}", end="")

            # Atualiza o cursor para a próxima iteração
            if cursor_mark == next_cursor_mark:  # Se não houver progresso no cursor, encerra
                break
            cursor_mark = next_cursor_mark

        elif response.status_code == 403:
            print(f"Erro 403: {response.text}")
            print("Aguardando 2 segundos antes de continuar...")
            time.sleep(2)

        else:
            print(f"Erro {response.status_code} ao buscar combinações.")
            print(f"Mensagem de erro: {response.text}")
            break

    print("\nProcessamento concluído!")

In [ ]:
# Inicializar banco de dados e executar busca
initialize_database()
fetch_combinations()

Iniciando processamento a partir do cursorMark: AoEoZnFrazAyMzc=
Progresso anterior: 871110 documentos processados (8.35%).
Processando... Cursor atual: AoEoZnJidjAyNTk=, Documentos processados: 894910, Progresso: 8.58%, Tempo decorrido: 0:05:18.490162

In [115]:
import pandas as pd

conn = sqlite3.connect(DB_PATH)
# Verificar se a tabela existe e carregar os dados
query_check = "SELECT name FROM sqlite_master WHERE type='table' AND name='Collections_Types';"
if not pd.read_sql_query(query_check, conn).empty:
    # Lê os dados da tabela no DataFrame do Pandas
    df = pd.read_sql_query("SELECT * FROM Collections_Types;", conn)

    # Verifica se há dados na tabela
    if not df.empty:
        print(f"Tabela 'Collections_Types' contém {len(df)} registros:")
    else:
        print("A tabela 'Collections_Types' está vazia.")
else:
    print("A tabela 'Collections_Types' não existe no banco de dados.")

# Fechar a conexão
conn.close()

A tabela 'Collections_Types' não existe no banco de dados.


In [ ]:
df

In [111]:
def drop_table():
    """
    Remove a tabela Collections_Types do banco de dados.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    try:
        # Dropar a tabela sem verificação prévia
        cursor.execute("DROP TABLE IF EXISTS Collections_Types;")
        conn.commit()
        print("Tabela 'Collections_Types' foi removida (se existia).")
    except Exception as e:
        print(f"Erro ao dropar a tabela: {e}")
    finally:
        conn.close()

In [113]:
# Exemplo de uso
drop_table()

Tabela 'Collections_Types' foi removida (se existia).
